In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import sys
print(sys.executable)

C:\Users\prana\anaconda3\envs\fresh_env\python.exe


In [3]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [4]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']
num_validation_samples=0.1* mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

def scale(image,label):
    image=tf.cast(image,tf.float32)  
    image/=255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)

##Shuffling

BUFFER_SIZE=1000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

##batching

BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_input,validation_output= next(iter(validation_data))

In [8]:
##Model outline 

input_size=784
output_size=10
hidden_layer_size=50

model=tf.keras.Sequential([ tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
            
])


In [26]:
#optimizer and loss function

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [28]:
NUM_EPOCHS=5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_input,validation_output),verbose=2)


Epoch 1/5
540/540 - 15s - 27ms/step - accuracy: 0.8780 - loss: 0.4220 - val_accuracy: 0.9367 - val_loss: 0.2141
Epoch 2/5
540/540 - 4s - 7ms/step - accuracy: 0.9479 - loss: 0.1791 - val_accuracy: 0.9450 - val_loss: 0.1737
Epoch 3/5
540/540 - 4s - 7ms/step - accuracy: 0.9592 - loss: 0.1363 - val_accuracy: 0.9552 - val_loss: 0.1428
Epoch 4/5
540/540 - 6s - 11ms/step - accuracy: 0.9666 - loss: 0.1140 - val_accuracy: 0.9573 - val_loss: 0.1341
Epoch 5/5
540/540 - 5s - 9ms/step - accuracy: 0.9712 - loss: 0.0950 - val_accuracy: 0.9595 - val_loss: 0.1291


In [30]:
#test the model
test_loss,test_accuracy=model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9626 - loss: 0.1206
